In [1]:
import spacy
from newsapi import NewsApiClient
API_KEY = 'd83a0fb9e3104414a3693c6c4fdab735'
COMPANY_NAME = 'exxon-mobile'
MULTIPLIER = 100

nlp = spacy.load('en_core_web_md')

RISK_WORDS = ['delayed', 'snag', 'effectiveness', 'safety', 'injured',
              'security', 'emergency', 'technical issue', 'safety cocerns']
RISK_TOKENS = []


def setup():
    for risk_word in RISK_WORDS:
        doc = nlp(risk_word)
        for risk_token in doc:
            RISK_TOKENS.append(risk_token)
        for risk_token in doc.ents:
            print(risk_token)
            RISK_TOKENS.append(risk_token)
    print(RISK_TOKENS)




def get_risk_score_token(token, company_name):
    score = 0
    for risk_token in RISK_TOKENS:
        score = max(score, token.similarity(risk_token))
    return int(MULTIPLIER * score)


def get_company_score_token(token, company_name):
    doc = nlp(company_name)
    company_token = doc[0]
    score = token.similarity(company_token)
    return int(MULTIPLIER * score)

def get_company_score_sentence(sentence, company_name):
    score = 0
    for token in sentence:
        score += get_company_score_token(token, company_name)
    score /= len(sentence)
    return score

def get_risk_score_sentence(sentence, company_name):
    score = 0
    for token in sentence:
        score += get_risk_score_token(token, company_name)
    score /= len(sentence)
    return score


def get_score_sentence(sentence, company_name):
    company_score = get_company_score_sentence(sentence, company_name)
    risk_score = get_risk_score_sentence(sentence, company_name)
    return (company_score * risk_score)


def get_score_text(text, company_name):
    doc = nlp(text)
    score = 0
    for sentence in doc.sents:
        score += get_score_sentence(sentence, company_name)
    score /= len(list(doc.sents))
    return score
setup()

2023-01-14 09:30:18.142844: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-14 09:30:18.235856: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-14 09:30:18.625113: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-14 09:30:18.625153: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

safety cocerns
[delayed, snag, effectiveness, safety, injured, security, emergency, technical, issue, safety, cocerns, safety cocerns]


In [2]:
text = '''
Microsoft was a fradulent and scheming company. They schemed their way to sell laptops through threat and deceit.
'''

In [3]:
get_score_text(text, 'microsoft')

/tmp/ipykernel_32381/449194880.py:37: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  score = token.similarity(company_token)
/tmp/ipykernel_32381/449194880.py:30: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  score = max(score, token.similarity(risk_token))


561.5428811454453